In [1]:
import random

import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from generator import Generator
from keras import metrics
import importlib
import inverse
from matplotlib.ticker import PercentFormatter

In [2]:
def custom_loss(y_true, y_pred):
    pass
class ConstrainedOutputLayer(tf.keras.layers.Layer):
    def __init__(self, M=None, weight=10, **kwargs):
        super(ConstrainedOutputLayer, self).__init__(**kwargs)
        # Initialize M as None to handle custom initialization in from_config
        if M is not None:
            self.M = tf.constant(M, dtype=tf.float32)
        else:
            self.M = None  # Will be set later during call or from_config
        self.weight = weight

    def call(self, y_out, x_in):
        if self.M is None:
            raise ValueError("Matrix M must be initialized.")
        # Apply the constraint logic
        constraint = tf.matmul(y_out, self.M) - x_in
        self.add_loss(self.weight * tf.reduce_mean(tf.square(constraint)))
        return y_out

    def get_config(self):
        # Get the base config and update it with custom arguments
        config = super(ConstrainedOutputLayer, self).get_config()
        # Serialize M as a list, and weight as is
        config.update({
            'M': self.M.numpy().tolist() if self.M is not None else None,
            'weight': self.weight
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Reconstruct the layer from the config
        M = tf.constant(config['M'], dtype=tf.float32) if config['M'] is not None else None
        weight = config['weight']
        return cls(M=M, weight=weight)


In [4]:
import tools.reportParser as parser
dataset, paths = parser.parse_all_files('data/reports/')

C:\Users\user\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\user\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\user\PycharmProjects\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are in

In [5]:
model = tf.keras.models.load_model('data/models/silica_CNN.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss,
                                                                                                        'ConstrainedOutputLayer': ConstrainedOutputLayer})
#model = tf.keras.models.load_model('data/models/silica_best_tyhanov.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})

In [6]:
cut_dataset = []
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
for i, iso in enumerate(dataset):
    if iso['p_adsorption'][0] > pressures[77] or iso['p_adsorption'][-1] < pressures[-10]:
        continue
    cut_dataset.append(iso)

In [7]:
pressures = np.load("data/initial kernels/Pressure_Silica.npy")[77:-10]
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

In [8]:
isotherms = []
for iso in cut_dataset:
    isotherms.append(np.interp(pressures, iso['p_adsorption'], iso['adsorption']))

In [9]:
isotherms[0].shape

(371,)

In [10]:
scales=[]
for i in range(len(isotherms)):
    scale = max(isotherms[i])
    isotherms[i] = isotherms[i] / scale
    scales.append(scale)

In [12]:
prediction = model.predict(np.array(isotherms))
#prediction = model.predict(np.array(text_x))

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [13]:

prediction = [i[:128] for i in prediction]

In [46]:
import random
i = random.randint(0, len(prediction)-1)
plt.plot(pore_widths, prediction[i], marker=".")
restored_isotherm = np.dot(kernel.T, prediction[i])
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".")
plt.plot(pressures, isotherms[i], marker="*")
plt.show()

In [215]:
def calculate_error(isotherm_real, restored_isotherm):
    defined_error = np.sum(np.abs(restored_isotherm - isotherm_real[77:-10]) / isotherm_real[77:-10]) / len(isotherm_real[77:-10])
    return defined_error

def calculate_roughness(psd):
    diff1 = np.diff(psd)/pore_widths[:-1]
    return np.sum(np.abs(np.diff(diff1)/pore_widths[:-2]))

error_lst = []
roughness_lst = []
for i in range(len(prediction)):
    # restored_isotherm = np.dot(kernel.T, prediction[i])
    # error_lst.append(calculate_error(restored_isotherm, isotherms[i])*100)
    restored_isotherm = np.dot(kernel.T, prediction[i])
    error_lst.append(calculate_error(restored_isotherm, np.array(text_x[i]))*100)
    roughness_lst.append(calculate_roughness(prediction[i]))

In [216]:
plt.hist(error_lst, bins=100)
plt.show()

In [217]:
model_tyhanov = tf.keras.models.load_model('data/models/silica_CNN3_transfer.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})
test = np.load("data/datasets/tyhanov_test.npz")
text_x = test["x_test"]
text_y = test["y_test"]
prediction_tyhanov = model_tyhanov.predict(np.array(text_x))
prediction_tyhanov = [i[:128] for i in prediction_tyhanov]
error_lst_tyhanov = []
roughness_lst_tyhanov = []
for i in range(len(text_x)):
    restored_isotherm = np.dot(kernel.T, prediction_tyhanov[i])
    error_lst_tyhanov.append(calculate_error(restored_isotherm, np.array(text_x[i]))*100)
    roughness_lst_tyhanov.append(calculate_roughness(prediction_tyhanov[i]))

3/3 [==============================] - 0s 3ms/step


In [218]:
import random
i = random.randint(0, len(prediction_tyhanov)-1)
plt.plot(pore_widths, prediction_tyhanov[i], marker=".")
restored_isotherm = np.dot(kernel.T, prediction_tyhanov[i])
print(roughness_lst_tyhanov[i])
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".")
plt.plot(pressures, text_x[i], marker="*")
plt.show()

0.2585616643927007


In [219]:
plt.hist(error_lst_tyhanov, bins=100)
plt.show()

In [220]:
error_lst_math = []
roughness_lst_math = []
prediction_math = []
math_kernel = kernel[:, 77:-10]
for i in range(len(isotherms)):
    math_PSD = inverse.fit_linear(isotherms[i], math_kernel, 1e-8).x
    restored_isotherm = np.dot(kernel.T, math_PSD)
    prediction_math.append(math_PSD)
    error_lst_math.append(calculate_error(restored_isotherm, isotherms[i])*100)
    roughness_lst_math.append(calculate_roughness(math_PSD))

In [221]:
i = 270 #random.randint(0, len(prediction_math)-1)
restored_isotherm = np.dot(kernel.T, prediction_math[i])
plt.plot(pore_widths, prediction_math[i], marker=".")
print(roughness_lst_math[i])
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".")
plt.plot(pressures, isotherms[i], marker="*")
plt.show()

0.005363562212461434


In [222]:
plt.hist(roughness_lst_math, bins=100)
plt.show()

In [223]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [224]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [225]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [226]:
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [227]:
len(isotherms)

609

In [228]:
i = 4
restored_isotherm = np.dot(kernel.T, prediction[i])
math_PSD = inverse.fit_linear(isotherms[i], math_kernel, alpha=0.001).x
restored_isotherm_math = np.dot(kernel.T, math_PSD)

plt.plot(pore_widths, prediction[i], marker=".", label="model")
plt.plot(pore_widths, math_PSD, marker=".", label="math")
plt.legend()
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".", label="model")
plt.plot(pressures, isotherms[i], marker="*", label="real")
plt.legend()
plt.show()

In [229]:
test = np.load("data/datasets/tyhanov_test.npz")
text_x = test["x_test"]
text_y = test["y_test"]
len(text_x)

92

In [230]:
i = 7
prediction_tyh = model_tyhanov.predict(text_x)
restored_isotherm = np.dot(kernel.T, prediction_tyh[i])
math_PSD = inverse.fit_linear(text_x[i], math_kernel, 0.001).x

plt.plot(pore_widths, prediction_tyh[i], marker=".", label="model")
plt.plot(pore_widths, math_PSD, marker=".", label="math")
plt.legend()
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".", label="model")
plt.plot(pressures, text_x[i], marker="*", label="real")
plt.legend()
plt.show()

3/3 [==============================] - 0s 4ms/step


In [241]:
exp_file_list = ["MCM-41", "SBA-15", "SBA-16", "MIL-101", "MIL-101_2", "DUT-49", "FDM-4", "PCN-333", "PCN-777",
                 "MIL-100"]

p_exp_list = []
n_s_exp_raw_list = []
for exp_file_name in exp_file_list:
    data = pd.read_csv(f"data/real/{exp_file_name}.txt", header=None)
    # p_exp_list.append(data.iloc[:,1].to_numpy())
    # n_s_exp_raw_list.append(data.iloc[:,3].to_numpy())
    p_exp_list.append(data.iloc[:, 1].to_numpy())
    n_s_exp_raw_list.append(data.iloc[:, 3].to_numpy())
# интерполируем экспериментальную изотерму под давления кернала
n_s_exp_list = []
for i in range(len(p_exp_list)):
    n_s_exp_list.append(np.interp(pressures, p_exp_list[i], n_s_exp_raw_list[i]))

In [232]:
for i in range(len(n_s_exp_list)):
    n_s_exp_list[i] = n_s_exp_list[i]/max(n_s_exp_list[i])

In [233]:
structures_prediction =  model_tyhanov.predict(np.array(n_s_exp_list))

1/1 [==============================] - 0s 167ms/step


In [234]:
structures_prediction = [i[:128] for i in structures_prediction]

In [247]:
i = 3
print(exp_file_list[i])
plt.plot(pore_widths[:-30], structures_prediction[i][:-30], marker=".", label="PSD")
plt.ylabel("Объем пор, $см^3$/$нм*г$")
plt.xlabel("Размер пор, $нм$")
plt.legend(loc='upper right')
plt.grid()
plt.show()

restored_isotherm = np.dot(math_kernel.T, structures_prediction[i])
plt.plot(pressures, restored_isotherm, label="Изотерма по полученному моделью распределению")
plt.plot(pressures, n_s_exp_list[i], label="Экспериментальная изотерма")
plt.ylabel("Адсорбция, $см^3$(НТД)/$г$")
plt.xlabel("Давление, $P/P_{0}$")
plt.legend(loc='upper left')
plt.grid()
plt.show()

MIL-101


In [236]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


x = np.linspace(0, 40, 1000)

kde_math = gaussian_kde(error_lst_math)
kde_model = gaussian_kde(error_lst)
kde_real = gaussian_kde(error_lst_tyhanov)

pdf_math = kde_math(x)
pdf_model = kde_model(x)
pdf_real = kde_real(x)

plt.plot(x, pdf_math, label='Классическое решение', color='blue')
plt.plot(x, pdf_model, label='Модель на основе искусственных распределений', color='orange')
plt.plot(x, pdf_real, label='Модель на основе реальных распределений', color='green')

plt.legend(loc='upper right')
#plt.ylabel(r'Ошибка описания изотермы 1/%')
plt.xlabel("Ошибка описания изотермы %")
plt.title("Распределения ошибок моделей (PDF)")
plt.grid()
plt.show()


In [237]:
roughness_max = max(max(roughness_lst_math), max(roughness_lst), max(roughness_lst_tyhanov))

x = np.linspace(0, 10, 1000)

kde_math = gaussian_kde(roughness_lst_math)
kde_model = gaussian_kde(roughness_lst)
kde_real = gaussian_kde(roughness_lst_tyhanov)

pdf_math = kde_math(x)
pdf_model = kde_model(x)
pdf_real = kde_real(x)

plt.plot(x, pdf_math, label='Классическое решение', color='blue')
plt.plot(x, pdf_model, label='Модель на основе искусственных распределений', color='orange')
plt.plot(x, pdf_real, label='Модель на основе реальных распределений', color='green')



plt.legend(loc='upper right')
#plt.ylabel(r'Ошибка описания изотермы 1/%')
plt.xlabel("roughness")
plt.title("Распределения roughness моделей (PDF)")
plt.grid()
plt.show()

In [248]:
error_min = min(error_lst_math)
roughness_min = min(roughness_lst_math)
error_max = max(error_lst_math)
roughness_max = max(roughness_lst_math)


X, Y = np.mgrid[error_min:error_max:300j, roughness_min:roughness_max:300j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([error_lst_math, roughness_lst_math])
kde = gaussian_kde(values)
Z = np.reshape(kde(positions).T, X.shape)

fig, ax = plt.subplots()
ax.imshow(np.rot90(Z), extent=[error_min, error_max, roughness_min, roughness_max], aspect='auto')
ax.plot(error_lst_math, roughness_lst_math, 'k.', markersize=3)
ax.set_xlim([error_min, error_max])
ax.set_ylim([roughness_min, roughness_max])
plt.xlabel("error %")
plt.ylabel("roughness")
plt.show()

In [239]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, edgecolor='royalblue', lw=0.5, rstride=8, cstride=8,
                alpha=0.3, cmap="hsv")
ax.set_xlabel('error %')
ax.set_ylabel('roughness')
plt.show()